In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("data/insurance.csv")
df.head()

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46
3,33,male,22.7,0,no,northwest,21984.47
4,32,male,28.9,0,no,northwest,3866.86


In [4]:
df.head()

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46
3,33,male,22.7,0,no,northwest,21984.47
4,32,male,28.9,0,no,northwest,3866.86


In [5]:
X=df.drop(labels=['expenses'],axis=1)
y= df[["expenses"]]

In [6]:
y

,expenses
0,16884.92
1,1725.55
2,4449.46
3,21984.47
4,3866.86
...,...
1333,10600.55
1334,2205.98
1335,1629.83
1336,2007.95


In [7]:
X

,age,sex,bmi,children,smoker,region
0,19,female,27.9,0,yes,southwest
1,18,male,33.8,1,no,southeast
2,28,male,33.0,3,no,southeast
3,33,male,22.7,0,no,northwest
4,32,male,28.9,0,no,northwest
...,...,...,...,...,...,...
1333,50,male,31.0,3,no,northwest
1334,18,female,31.9,0,no,northeast
1335,18,female,36.9,0,no,southeast
1336,21,female,25.8,0,no,southwest


In [8]:
# Segregating numerical and categorical variables
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [10]:
categorical_cols


Index(['sex', 'smoker', 'region'], dtype='object')

In [12]:
# Define the custom ranking for each ordinal variable
sex_categories = ['female','male']
smoker_categories = ['no','yes']
region_categories = ['southwest', 'southeast', 'northwest', 'northeast']

In [13]:
df.head()

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46
3,33,male,22.7,0,no,northwest,21984.47
4,32,male,28.9,0,no,northwest,3866.86


In [14]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [15]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[sex_categories,smoker_categories,region_categories])),
    ('scaler',StandardScaler())
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])


In [16]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=30)

In [17]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [18]:
X_train.head()

,num_pipeline__age,num_pipeline__bmi,num_pipeline__children,cat_pipeline__sex,cat_pipeline__smoker,cat_pipeline__region
0,-1.261206,-0.760093,-0.925046,1.006431,-0.511327,1.411177
1,-0.978336,0.495641,2.437452,1.006431,-0.511327,-0.417005
2,-0.624748,2.208006,0.756203,1.006431,-0.511327,-0.417005
3,1.284628,-0.450236,-0.084422,1.006431,-0.511327,-0.417005
4,-0.766183,-1.086257,1.596828,1.006431,1.955695,-0.417005


In [19]:
X_test.head()

,num_pipeline__age,num_pipeline__bmi,num_pipeline__children,cat_pipeline__sex,cat_pipeline__smoker,cat_pipeline__region
0,0.789604,0.267326,-0.084422,1.006431,1.955695,1.411177
1,-0.624748,0.120552,-0.084422,1.006431,-0.511327,-1.331096
2,-0.271160,-0.580702,-0.084422,1.006431,-0.511327,-1.331096
3,-0.483313,-2.097368,0.756203,-0.993610,1.955695,0.497086
4,0.931040,0.984889,-0.925046,1.006431,-0.511327,-1.331096


In [20]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [21]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [22]:
regression.coef_

array([[3811.90482516, 1955.00989449,  512.09623573,  -71.33973587,
        9570.52652349,  274.9300558 ]])

In [23]:
regression.intercept_


array([13182.01547009])

In [24]:
import numpy as np

In [25]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [26]:
#train multiple model
##model evaluation
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    "Ridge":Ridge(),
    "ElasticNet":ElasticNet(),
    "decision_tree":DecisionTreeRegressor()
}
trained_model_list=[]
model_list=[]
r2_list=[]
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 5858.705669057534
MAE: 4028.5289830492975
R2 score 76.31627791214297


Lasso
Model Training Performance
RMSE: 5858.791513025547
MAE: 4028.422188888716
R2 score 76.3155838613778


Ridge
Model Training Performance
RMSE: 5858.9108205057455
MAE: 4029.6366008713617
R2 score 76.31461924033793


ElasticNet
Model Training Performance
RMSE: 6871.765171439434
MAE: 4965.526451058497
R2 score 67.4175891649489


decision_tree
Model Training Performance
RMSE: 6896.942586685302
MAE: 3267.900746268657
R2 score 67.17839481310641




In [28]:
regression.predict()

array([[ 1.83722161e+03],
       [ 6.58977034e+03],
       [ 1.04243429e+04],
       [ 1.20753287e+04],
       [ 2.74860679e+04],
       [ 1.01150592e+04],
       [ 7.08105303e+03],
       [ 1.16967840e+04],
       [ 5.08815733e+03],
       [ 8.17295317e+03],
       [ 3.28384383e+03],
       [ 9.77558950e+03],
       [ 6.31876431e+03],
       [ 3.55945884e+04],
       [ 2.68385589e+04],
       [ 6.12306747e+03],
       [ 2.39024482e+03],
       [ 1.04310783e+04],
       [-3.66824185e+02],
       [ 1.62815232e+04],
       [ 2.47138914e+04],
       [ 2.72357403e+02],
       [ 3.01926363e+03],
       [ 3.56047275e+04],
       [ 1.08315045e+04],
       [ 9.63684589e+03],
       [ 4.12266965e+03],
       [ 2.51338013e+03],
       [ 1.52963695e+04],
       [ 1.07297051e+04],
       [ 4.01417238e+03],
       [ 4.01196432e+04],
       [ 2.56543100e+03],
       [ 2.92792292e+04],
       [ 4.04336910e+04],
       [ 3.67016726e+04],
       [ 8.39101016e+03],
       [ 3.92003697e+04],
       [ 9.5